In [ ]:
!pip install -q --upgrade "numpy>=2.2.0,<3.0.0" "scipy>=1.13.0" networkx==3.2.1
!pip install -q ultralytics openvino-dev pillow tqdm requests

In [ ]:
from openvino.runtime import Core
print('OpenVINO version:', Core().get_versions('CPU')['CPU'])

In [ ]:
import time, zipfile, requests, numpy as np
from io import BytesIO
from pathlib import Path
from ultralytics import YOLO
from openvino.runtime import Core
from PIL import Image

DATA_ROOT_640 = Path("datasets/coco8_640")
COCO8_YAML_640 = "coco8_640.yaml"
yaml_url_640  = ("https://raw.githubusercontent.com/ultralytics/ultralytics/"
             "main/ultralytics/cfg/datasets/coco8.yaml")
zip_url_640   = ("https://github.com/ultralytics/assets/releases/download/"
             "v0.0.0/coco8.zip")

if not DATA_ROOT_640.exists():
    with zipfile.ZipFile(BytesIO(requests.get(zip_url_640, timeout=60).content)) as zf:
        zf.extractall(DATA_ROOT_640.parent)
if not Path(COCO8_YAML_640).exists():
    yaml_640 = requests.get(yaml_url_640, timeout=10).text
    yaml_640 = yaml_640.replace("../datasets/coco8", str(DATA_ROOT_640.name))
    Path(COCO8_YAML_640).write_text(yaml_640)

MODEL_640 = "yolo11x.pt"
yolo_640  = YOLO(MODEL_640)

ov_dir_fp32_640 = yolo_640.export(format="openvino", project="exported_models_640", name=f"{Path(MODEL_640).stem}_fp32_640")
ov_dir_int8_640 = yolo_640.export(format="openvino", int8=True, data=COCO8_YAML_640, project="exported_models_640", name=f"{Path(MODEL_640).stem}_int8_640")

core_640 = Core()
def load_640(ov_dir_path_str):
    ov_dir = Path(ov_dir_path_str)

    xml_files = list(ov_dir.glob(f"{Path(MODEL_640).stem}.xml"))
    if not xml_files:
        xml_files = list(ov_dir.glob("*.xml"))
    if not xml_files:
        raise FileNotFoundError(f"No .xml file in {ov_dir}")
    xml = xml_files[0]
    comp = core_640.compile_model(xml.as_posix(), "CPU")
    return comp, comp.input(0).get_any_name()

fp32_comp_640, fp32_in_640 = load_640(ov_dir_fp32_640)
int8_comp_640, int8_in_640 = load_640(ov_dir_int8_640)

img_640 = Image.open(BytesIO(requests.get(
      "https://ultralytics.com/images/bus.jpg", timeout=10).content)).resize((640, 640))
arr_640 = np.transpose(np.array(img_640)[..., ::-1], (2, 0, 1))
arr_fp32_640 = (arr_640[np.newaxis] / 255).astype(np.float32)
arr_int8_640 =  arr_640[np.newaxis].astype(np.uint8)

def bench_640(compiled, frame, iters=100):
    compiled([frame])
    t0 = time.perf_counter()
    for _ in range(iters):
        compiled([frame])
    return (time.perf_counter() - t0) / iters * 1000

fp32_ms_640 = bench_640(fp32_comp_640, arr_fp32_640)
int8_ms_640 = bench_640(int8_comp_640, arr_int8_640)

print(f"OpenVINO FP32 (640): {fp32_ms_640:.2f} ms")
print(f"OpenVINO INT8 (640): {int8_ms_640:.2f} ms")

In [ ]:
import os, time, numpy as np, torch, requests
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
from openvino.runtime import Core
from io import BytesIO

MODEL_PT_C4    = "yolo11x.pt"
OV_INT8_DIR_C4 = Path("exported_models_640") / f"{Path(MODEL_PT_C4).stem}_int8_640"
IMG_URL_C4     = "https://ultralytics.com/images/bus.jpg"
ITERS_C4       = 100

def load_image_c4(url, size=640):
    img_c4 = Image.open(BytesIO(requests.get(url, timeout=10).content)).convert("RGB")
    img_c4 = img_c4.resize((size, size))
    arr_c4 = np.array(img_c4)[..., ::-1]
    return np.transpose(arr_c4, (2, 0, 1)).copy()

def bench_c4(fn, reps=100):
    fn()
    t0 = time.perf_counter()
    for _ in range(reps):
        fn()
    return (time.perf_counter() - t0) / reps * 1000

if Path(MODEL_PT_C4).exists():
    torch.set_num_threads(os.cpu_count() or 1)
    pt_model_c4  = YOLO(MODEL_PT_C4).model.cpu().eval()
    img_arr_c4   = load_image_c4(IMG_URL_C4)
    pt_tensor_c4 = torch.from_numpy(img_arr_c4).unsqueeze(0).float() / 255

    @torch.inference_mode()
    def run_pt_c4():
        _ = pt_model_c4(pt_tensor_c4)

    pt_ms_c4 = bench_c4(run_pt_c4, ITERS_C4)
    print(f"PyTorch FP32 (640): {pt_ms_c4:.2f} ms")
else:
    print(f"{MODEL_PT_C4} not found for PyTorch benchmark.")


if OV_INT8_DIR_C4.exists():
    core_c4 = Core()
    core_c4.set_property({"INFERENCE_NUM_THREADS": os.cpu_count() or 1})

    xml_files_c4 = list(OV_INT8_DIR_C4.glob(f"{Path(MODEL_PT_C4).stem}.xml"))
    if not xml_files_c4: xml_files_c4 = list(OV_INT8_DIR_C4.glob("*.xml"))

    if xml_files_c4:
        xml_path_c4 = xml_files_c4[0]
        ov_comp_c4  = core_c4.compile_model(xml_path_c4.as_posix(),
                                      "CPU",
                                      {"PERFORMANCE_HINT": "THROUGHPUT"})

        img_arr_ov_c4 = load_image_c4(IMG_URL_C4)
        ov_tensor_c4 = img_arr_ov_c4[np.newaxis].astype(np.uint8)

        def run_ov_c4():
            _ = ov_comp_c4([ov_tensor_c4])

        ov_ms_c4 = bench_c4(run_ov_c4, ITERS_C4)
        print(f"OpenVINO INT8 (640, from {MODEL_PT_C4}): {ov_ms_c4:.2f} ms")
    else:
        print(f"No XML file found in {OV_INT8_DIR_C4}")
else:
    print(f"OpenVINO INT8 directory {OV_INT8_DIR_C4} not found.")

In [ ]:

import time, zipfile, requests, numpy as np
from io import BytesIO
from pathlib import Path
from ultralytics import YOLO
from openvino.runtime import Core
from PIL import Image

INPUT_SIZE_C5 = 320
MODEL_NAME_C5 = "yolo11x.pt"
model_stem_c5 = Path(MODEL_NAME_C5).stem

DATA_ROOT_C5 = Path(f"datasets/coco8_generic_for_320calib")
COCO_YAML_FILENAME_C5 = f"coco8_generic_for_320calib.yaml"
yaml_url_c5  = ("https://raw.githubusercontent.com/ultralytics/ultralytics/"
             "main/ultralytics/cfg/datasets/coco8.yaml")
zip_url_c5   = ("https://github.com/ultralytics/assets/releases/download/"
             "v0.0.0/coco8.zip")

generic_coco8_path_in_yaml = "./datasets/coco8"
actual_coco8_data_dir_for_dl = Path("datasets/coco8")

if not actual_coco8_data_dir_for_dl.exists():
    with zipfile.ZipFile(BytesIO(requests.get(zip_url_c5, timeout=60).content)) as zf:
        zf.extractall(actual_coco8_data_dir_for_dl.parent)

if not Path(COCO_YAML_FILENAME_C5).exists():
    coco8_yaml_content_original = requests.get(yaml_url_c5, timeout=10).text
    coco8_yaml_content_adjusted = coco8_yaml_content_original.replace(
        "../datasets/coco8", generic_coco8_path_in_yaml
    )
    Path(COCO_YAML_FILENAME_C5).write_text(coco8_yaml_content_adjusted)

yolo_c5 = YOLO(MODEL_NAME_C5)
export_project_320 = "exported_models_320"

ov_dir_fp32_path_str_c5 = yolo_c5.export(
    format="openvino",
    imgsz=INPUT_SIZE_C5,
    half=False,
    project=export_project_320,
    name=f"{model_stem_c5}_fp32_{INPUT_SIZE_C5}_openvino"
)

ov_dir_int8_path_str_c5 = yolo_c5.export(
    format="openvino",
    imgsz=INPUT_SIZE_C5,
    int8=True,
    data=COCO_YAML_FILENAME_C5,
    project=export_project_320,
    name=f"{model_stem_c5}_int8_{INPUT_SIZE_C5}_openvino"
)

core_c5 = Core()
def load_openvino_model_c5(ov_model_dir_str: str, base_model_stem: str):
    potential_model_dir = Path(ov_model_dir_str)
    xml_files = list(potential_model_dir.glob(f"{base_model_stem}.xml"))
    if not xml_files:
        xml_files = list(potential_model_dir.glob("*.xml"))
        raise FileNotFoundError(f"No .xml file in {ov_model_dir_str}")
    xml_path = xml_files[0]
    compiled_model = core_c5.compile_model(xml_path.as_posix(), "CPU")
    return compiled_model, compiled_model.input(0).get_any_name()

fp32_compiled_model_c5, _ = load_openvino_model_c5(ov_dir_fp32_path_str_c5, model_stem_c5)
int8_compiled_model_c5, _ = load_openvino_model_c5(ov_dir_int8_path_str_c5, model_stem_c5)

img_c5 = Image.open(BytesIO(requests.get(
      "https://ultralytics.com/images/bus.jpg", timeout=10).content)).resize((INPUT_SIZE_C5, INPUT_SIZE_C5))
arr_c5 = np.transpose(np.array(img_c5)[..., ::-1], (2, 0, 1))
arr_fp32_320_c5 = (arr_c5[np.newaxis] / 255.0).astype(np.float32)
arr_int8_320_c5 = arr_c5[np.newaxis].astype(np.uint8)

def bench_c5(compiled_model, input_frame, iters=100):
    compiled_model([input_frame])
    t0 = time.perf_counter()
    for _ in range(iters):
        compiled_model([input_frame])
    return (time.perf_counter() - t0) / iters * 1000

fp32_latency_ms_c5 = bench_c5(fp32_compiled_model_c5, arr_fp32_320_c5)
int8_latency_ms_c5 = bench_c5(int8_compiled_model_c5, arr_int8_320_c5)

print(f"OpenVINO FP32 ({INPUT_SIZE_C5}): {fp32_latency_ms_c5:.2f} ms")
print(f"OpenVINO INT8 ({INPUT_SIZE_C5}): {int8_latency_ms_c5:.2f} ms")
if int8_latency_ms_c5 > 0 :
    speedup_c5 = fp32_latency_ms_c5 / int8_latency_ms_c5
    print(f"Speed-up ({INPUT_SIZE_C5}): {speedup_c5:.2f}x")